<a href="https://colab.research.google.com/github/pitfisher/miem_face_recognition/blob/main/no_rotated_comparing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install deepface
!pip install matplotlib
import cv2
import time
import os
import glob
from deepface import DeepFace
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
folder_path = '/content/drive/MyDrive/face_dataset/dataset/upd_face_dataset/center/center'
tmp_folder = "tmp"
os.makedirs(tmp_folder, exist_ok=True)

In [ ]:
model_names = [
    #"VGG-Face",
    #"Facenet",
    #"Facenet512",
    "OpenFace",
    #"DeepFace",
    "ArcFace",
    "SFace"
]

In [ ]:
model_resolutions = {
    "VGG-Face": 224,
    "Facenet": 220,
    "Facenet512": 160,
    "OpenFace": 96,
    "DeepFace": 152,
    "ArcFace": 112,
    "SFace": 112,
}

In [ ]:
def crop_to_square(image, face_info, border_pixels=250):
    x, y, w, h = face_info['facial_area']['x'], face_info['facial_area']['y'], face_info['facial_area']['w'], face_info['facial_area']['h']

    square_size = max(w, h) + 2 * border_pixels
    new_x = max(0, x + w // 2 - square_size // 2)
    new_y = max(0, y + h // 2 - square_size // 2)

    cropped_img = image[new_y: new_y + square_size, new_x: new_x + square_size]

    return cropped_img

In [ ]:
def resize_image(image, new_resolution):
    resized_image = cv2.resize(image, new_resolution)
    return resized_image

In [ ]:
for model_name in model_names:
    resolution = model_resolutions[model_name]
    df_list = []

    files = glob.glob(os.path.join(folder_path, "**", "*.JPG"), recursive=True)

    for img_path in files:
        img = cv2.imread(img_path)
        faces = DeepFace.extract_faces(img_path=img_path, detector_backend="retinaface")

        for face_info in faces:
            cropped_face = crop_to_square(img, face_info)
            resized_img = resize_image(cropped_face, (resolution, resolution))
            rel_path = os.path.relpath(img_path, folder_path)
            rel_path = rel_path.replace(os.path.sep, "_")
            output_path = os.path.join(tmp_folder, f"{rel_path[:-4]}_{resolution}.jpg")
            cv2.imwrite(output_path, resized_img)

    image_files = [f for f in os.listdir(tmp_folder) if f.endswith(f'_{resolution}.jpg')]

    for filename1 in tqdm(image_files, desc=f'Processing with {model_name}', unit='image'):
        img_path1 = os.path.join(tmp_folder, filename1)

        for filename2 in image_files:
            img_path2 = os.path.join(tmp_folder, filename2)

            result = DeepFace.verify(
                img1_path=img_path1,
                img2_path=img_path2,
                enforce_detection=True,
                detector_backend="retinaface",
                model_name=model_name,
                distance_metric="euclidean_l2"
            )

            img1_name = os.path.splitext(filename1)[0].rsplit('_', 1)[0]
            img2_name = os.path.splitext(filename2)[0].rsplit('_', 1)[0]

            df_list.append(pd.DataFrame({'img_1': [img1_name], 'img_2': [img2_name], 'distance': [result['distance']], 'verified': [result['verified']]}))

        df = pd.concat(df_list, ignore_index=True)

    csv_file_path = os.path.join(tmp_folder, f'{model_name}_comparison_results.csv')
    df.to_csv(csv_file_path, index=False)

Processing with OpenFace:   1%|▏         | 1/72 [15:04<17:50:22, 904.55s/image]


KeyboardInterrupt: 